In [1]:
import sys
sys.path.append("../../../../../Tangram")

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import tangram2 as tg2
import tangram as tg
from sklearn.metrics import jaccard_score

/gstore/home/huangh83/miniconda3/envs/tangram/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /gstore/home/huangh83/miniconda3/envs/tangram/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warn(f"Failed to load image Python extension: {e}")


In [3]:
# Load data
path = '/gstore/data/resbioai/tangram2_data/telegraph/res/ccc_data/original_dataset/'
ad_sc = sc.read_h5ad(path + 'scc_new.h5ad')

# Select patient and generate cell mix data
patient_group = ['P2', 'P4', 'P5', 'P6', 'P9', 'P10']
patient = patient_group[0]
ad_sc_sample = ad_sc[ad_sc.obs['patient'] == patient]

# Define label column
label_used = 'level2_celltype_mod' # Assuming this is the intended label based on common usage in similar analyses

ad_sp, ad_sc_paired = tg2.evalkit.datagen.cellmix.cellmix.cellmix(ad_sc_sample,
              n_spots = 100,
              n_cells_per_spot = 10,
              n_types_per_spot = 3,
              label_col=label_used,
              encode_spatial = True,
              resample = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
              )

In [4]:
# Get noise levels
noise_level = ['no_noise']
for key in ad_sp.layers:
    noise_level.append(key)

# Create ground truth mapping
ground_truth = pd.DataFrame(index=ad_sp.obs.index, columns=ad_sc_paired.obs.index).fillna(0)
for i in range(len(ad_sp.uns['cellmix_cell_map_mp']['row_self'])):
    ground_truth.iloc[ad_sp.uns['cellmix_cell_map_mp']['row_self'][i], ad_sp.uns['cellmix_cell_map_mp']['row_target'][i]] = 1

In [5]:
# Save generated data
# ad_sp.write_h5ad('/gstore/data/resbioai/tangram2_data/telegraph/res/mapping/100_spot/ad_sp_' + patient + '.h5ad')
# ad_sc_paired.write_h5ad('/gstore/data/resbioai/tangram2_data/telegraph/res/mapping/100_spot/ad_sc_' + patient + '.h5ad')

In [6]:
# Initialize results dictionaries
result_tg2integrate = {}
result_tg2cell = {}
result_tg1cell = {}
result_spaotsc = {}
result_moscot = {}
result_argmax = {}
result_random = {}

In [7]:
# Define mapping methods for easier iteration
mapping_methods = {
    'argmax': {'method': tg2.evalkit.met.map_methods.ArgMaxCorrMap},
    'random': {'method': tg2.evalkit.met.map_methods.RandomMap},
}

for noise in noise_level:
    # Prepare AnnData objects for mapping
    ad_sc_mapping = ad_sc_paired.copy()
    ad_sp_mapping = ad_sp.copy()
    if noise != 'no_noise':
        ad_sp_mapping.X = ad_sp_mapping.layers[noise]

    # Run ArgMaxCorrMap and RandomMap
    for method_name, method_dict in mapping_methods.items():
        input_dict = tg2.evalkit.met.utils.adatas_to_input(
            {'from': ad_sc_mapping, 'to': ad_sp_mapping},
            categorical_labels={'from': [label_used]}
        )
        tg2.evalkit.met.workflows.Workflow({'map': method_dict}).run(input_dict)
        if method_name == 'argmax':
            result_argmax[noise] = input_dict['T'].copy()
        elif method_name == 'random':
            result_random[noise] = input_dict['T'].copy()

    # Run Tangram2Map (tg2_integrate)
    input_dict_tg2integrate = tg2.evalkit.met.utils.adatas_to_input(
        {'from': ad_sc_mapping, 'to': ad_sp_mapping},
        categorical_labels={'from': [label_used]}
    )
    tg2.evalkit.met.pp.StandardTangram2.run(input_dict_tg2integrate)
    map_res_tg2integrate = tg2.evalkit.met.map_methods.Tangram2Map.run(
        input_dict_tg2integrate,
        num_epochs=1000,
        density_prior='uniform',
    )
    input_dict_tg2integrate.update(map_res_tg2integrate)
    tg2.evalkit.met.pp.StandardScanpy.run(input_dict_tg2integrate, target_objs=['X_from'])
    input_dict_tg2integrate['w'].index = input_dict_tg2integrate['w']['cell_type']
    result_tg2integrate[noise] = input_dict_tg2integrate['T'].copy()

    # Run Tangram2 (tg2.mapping.map_cells_to_space, tg2_cell)
    ad_sc_mapping_tg2 = ad_sc_paired.copy()
    ad_sp_mapping_tg2 = ad_sp.copy()
    if noise != 'no_noise':
        ad_sp_mapping_tg2.X = ad_sp_mapping_tg2.layers[noise]
    tg2.mapping.pp_adatas(ad_sc_mapping_tg2, ad_sp_mapping_tg2)
    ad_map_tg2 = tg2.mapping.map_cells_to_space(
        ad_sc_mapping_tg2, ad_sp_mapping_tg2,
        mode="cells",
        device='cuda:0',
        density_prior='uniform',
    )
    result_tg2cell[noise] = ad_map_tg2.to_df().T

    # Run Tangram1 (tan.map_cells_to_space, tg1_cell)
    ad_sc_mapping_tg = ad_sc_paired.copy()
    ad_sp_mapping_tg = ad_sp.copy()
    if noise != 'no_noise':
        ad_sp_mapping_tg.X = ad_sp_mapping_tg.layers[noise]
    tg.pp_adatas(ad_sc_mapping_tg, ad_sp_mapping_tg)
    ad_map_tg = tg.map_cells_to_space(
        ad_sc_mapping_tg, ad_sp_mapping_tg,
        mode="cells",
        device='cuda:0',
        density_prior='uniform',
    )
    result_tg1cell[noise] = ad_map_tg.to_df().T

    # Run spaOTsc
    # input_dict_spaotsc = tg2.evalkit.met.utils.adatas_to_input(
    #     {'from': ad_sc_mapping, 'to': ad_sp_mapping},
    #     categorical_labels={'from': [label_used]}
    # )
    # wf_spaOT_setup = {
    #     'pp': {'method': tg2.evalkit.met.pp.StandardSpaOTsc},
    #     'map': {'method': tg2.evalkit.met.map.SpaOTscMap, 'params': {'num_epochs': 1000, 'genes': None}},
    #     'pred': {'method': tg2.evalkit.met.pred_methods.MoscotPred},
    # }
    # wf_spaOT = tg2.evalkit.met.workflows.Workflow(wf_spaOT_setup)
    # wf_spaOT.run(input_dict_spaotsc)
    # result_spaotsc[noise] = input_dict_spaotsc['T']

    # Run MOSCOT
    input_dict_moscot = tg2.evalkit.met.utils.adatas_to_input(
        {'from': ad_sc_mapping, 'to': ad_sp_mapping},
        categorical_labels={'from': [label_used]}
    )
    wf_moscot_setup = {
        'pp': {'method': tg2.evalkit.met.pp.StandardMoscot},
        'map': {'method': tg2.evalkit.met.map.MoscotMap, 'params': {'num_epochs': 1000, 'genes': None}},
        'pred': {'method': tg2.evalkit.met.pred_methods.MoscotPred},
    }
    wf_moscot = tg2.evalkit.met.workflows.Workflow(wf_moscot_setup)
    wf_moscot.run(input_dict_moscot)
    result_moscot[noise] = input_dict_moscot['T']



INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 19775 genes and uniform density_prior in clusters mode...
INFO:root:Printing scores every 100 epochs.


Set Solid Seed
Set Solid Seed
Score: 0.919, KL reg: 3.121, Entropy reg: -7.213
Score: 0.973, KL reg: 3.091, Entropy reg: -5.626
Score: 0.974, KL reg: 3.091, Entropy reg: -5.487
Score: 0.974, KL reg: 3.091, Entropy reg: -5.454
Score: 0.974, KL reg: 3.091, Entropy reg: -5.429
Score: 0.974, KL reg: 3.091, Entropy reg: -5.416
Score: 0.974, KL reg: 3.091, Entropy reg: -5.408
Score: 0.974, KL reg: 3.091, Entropy reg: -5.402
Score: 0.974, KL reg: 3.091, Entropy reg: -5.398


INFO:root:Renormalizing Single cell data


Score: 0.974, KL reg: 3.091, Entropy reg: -5.397


INFO:root:Begin training with 19775 genes and uniform density_prior in cells mode after renormalization
INFO:root:Printing scores every 100 epochs.


...

Score: 0.928, KL reg: 0.001, Entropy reg: -4062.445
Score: 0.998, KL reg: 0.000, Entropy reg: -509.294
Score: 0.998, KL reg: 0.000, Entropy reg: -419.453
Score: 0.998, KL reg: 0.000, Entropy reg: -392.341
Score: 0.998, KL reg: 0.000, Entropy reg: -378.882
Score: 0.998, KL reg: 0.000, Entropy reg: -371.487
Score: 0.998, KL reg: 0.000, Entropy reg: -367.203
Score: 0.998, KL reg: 0.000, Entropy reg: -364.253
Score: 0.998, KL reg: 0.000, Entropy reg: -362.716
Score: 0.998, KL reg: 0.000, Entropy reg: -361.930


INFO:root:Saving results..
INFO:root:19777 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:19777 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 19777 genes and uniform density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.476, KL reg: 0.001
Score: 0.993, KL reg: 0.003
Score: 0.993, KL reg: 0.003
Score: 0.993, KL reg: 0.003
Score: 0.993, KL reg: 0.002
Score: 0.993, KL reg: 0.002
Score: 0.994, KL reg: 0.002
Score: 0.994, KL reg: 0.002
Score: 0.994, KL reg: 0.002
Score: 0.994, KL reg: 0.002


INFO:root:Saving results..


ModuleNotFoundError: No module named 'jax.extend'

In [ ]:
# Calculate Jaccard Scores
result_df = pd.DataFrame(index=noise_level)
for noise in noise_level:
    result_df.loc[noise, 'tg1_cell'] = jaccard_score((result_tg1cell[noise] == result_tg1cell[noise].max()).astype(int).values.flatten(), ground_truth.values.flatten())
    result_df.loc[noise, 'tg2_cell'] = jaccard_score((result_tg2cell[noise] == result_tg2cell[noise].max()).astype(int).values.flatten(), ground_truth.values.flatten())
    result_df.loc[noise, 'tg2_integrate'] = jaccard_score((result_tg2integrate[noise] == result_tg2integrate[noise].max()).astype(int).values.flatten(), ground_truth.values.flatten())
    result_df.loc[noise, 'moscot'] = jaccard_score((result_moscot[noise] == result_moscot[noise].max()).astype(int).values.flatten(), ground_truth.values.flatten())
    result_df.loc[noise, 'spaotsc'] = jaccard_score((result_spaotsc[noise] == result_spaotsc[noise].max()).astype(int).values.flatten(), ground_truth.values.flatten())
    result_df.loc[noise, 'argmax'] = jaccard_score((result_argmax[noise] == result_argmax[noise].max()).astype(int).values.flatten(), ground_truth.values.flatten())
    result_df.loc[noise, 'random'] = jaccard_score((result_random[noise] == result_random[noise].max()).astype(int).values.flatten(), ground_truth.values.flatten())

In [ ]:
print(result_df)

In [ ]:
result_df.to_csv('/gstore/data/resbioai/tangram2_data/telegraph/res/mapping/100_spot/' + patient + '_result.csv')


In [6]:
ad_sc = sc.read_h5ad(path + 'scc_new.h5ad')

In [7]:
ad_sc

AnnData object with n_obs × n_vars = 35022 × 32738
    obs: 'nCount_RNA', 'nFeature_RNA', 'patient', 'tum.norm', 'level1_celltype', 'level2_celltype', 'level3_celltype', 'level2_celltype_mod'
    obsm: 'X_cellplm_0', 'X_cellplm_1', 'X_cellplm_2', 'X_cellplm_3', 'X_cellplm_4'

In [8]:
patient_group = ['P2', 'P4', 'P5', 'P6', 'P9', 'P10']
patient = patient_group[0]
ad_sc_sample = ad_sc[ad_sc.obs['patient'] == patient]

In [33]:
ad_sp , ad_sc_paired  = tg.datagen.cellmix.cellmix.cellmix(ad_sc_sample,
              n_spots = 100,
              n_cells_per_spot = 10,
              n_types_per_spot = 3,
              label_col=label_used,
              encode_spatial = True,
              resample = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
              #n_interactions = 0, # set number of interactions to 2
              #signal_effect_scaling = (float(direc.split('_')[1]), float(direc.split('_')[2])), #effect_direction
              #signal_effect_base = 0.995,
              #effect_direction = direc.split('_')[0]
              )

In [34]:
noise_level = ['no_noise']
for key in ad_sp.layers:
    noise_level.append(key)

In [35]:
ground_truth = pd.DataFrame(index = ad_sp.obs.index, columns = ad_sc_paired.obs.index )
ground_truth = ground_truth.fillna(0)

for i in range(len(ad_sp.uns['cellmix_cell_map_mp']['row_self'])):
    ground_truth.iloc[ad_sp.uns['cellmix_cell_map_mp']['row_self'][i], ad_sp.uns['cellmix_cell_map_mp']['row_target'][i]] = 1

In [36]:
ad_sp.write_h5ad('/gstore/data/resbioai/tangram2_data/telegraph/res/mapping/100_spot/ad_sp_' + patient + '.h5ad')
ad_sc_paired.write_h5ad('/gstore/data/resbioai/tangram2_data/telegraph/res/mapping/100_spot/ad_sc_' + patient + '.h5ad')

In [37]:
result_tg2integrate = dict()
result_tg2cell = dict()
result_tg1cell = dict()
result_spaotsc = dict()
result_moscot = dict()
result_argmax = dict()
result_random = dict()

for i in range(len(ad_sc_group)):    
    noise = noise_level[0]

    ad_sc_mapping = ad_sc_paired.copy()
    ad_sp_mapping = ad_sp.copy()
    
    label_col = label_used
    input_dict_1 = tg.met.utils.adatas_to_input({'from':ad_sc_mapping, 'to':ad_sp_mapping}, # provide the data to be used
                                                    categorical_labels={'from':[label_col]}, # include cluster labels in the design matrix
                                                  )
    argmax_wf = {#'pp' : dict(method = tg.methods.preprocess.StandardTangramV1),
                     'map': dict(method = tg.met.map_methods.ArgMaxCorrMap),
                    }
    
    #argmax_wf = dict(method = tg.met.map_methods.ArgMaxCorrMap)
    
    tg.met.workflows.Workflow(argmax_wf).run(input_dict_1)
    result_argmax[noise] = input_dict_1['T'].copy()

    ad_sc_mapping = ad_sc_paired.copy()
    ad_sp_mapping = ad_sp.copy()
    
    input_dict_1 = tg.met.utils.adatas_to_input({'from':ad_sc_mapping, 'to':ad_sp_mapping}, # provide the data to be used
                                                    categorical_labels={'from':[label_col]}, # include cluster labels in the design matrix
                                                  )
    random_wf = {#'pp' : dict(method = tg.methods.preprocess.StandardTangramV1),
                     'map': dict(method = tg.met.map_methods.RandomMap),
                    }
    
    #argmax_wf = dict(method = tg.met.map_methods.ArgMaxCorrMap)
    
    tg.met.workflows.Workflow(random_wf).run(input_dict_1)
    result_random[noise] = input_dict_1['T'].copy()


    
    ad_sc_mapping = ad_sc_paired.copy()
    ad_sp_mapping = ad_sp.copy()
    
    input_dict_1 = tg.met.utils.adatas_to_input({'from':ad_sc_mapping, 'to':ad_sp_mapping}, # provide the data to be used
                                                    categorical_labels={'from':[label_col]}, # include cluster labels in the design matrix
                                                  )
    
    tg.met.pp.StandardTangramV2.run(input_dict_1)
    
    map_res_1 = tg.met.map_methods.TangramV2Map.run(input_dict_1,
                                                num_epochs = 1000,
                                                density_prior='uniform',
                                             )
    
    input_dict_1.update(map_res_1)
    tg.met.pp.StandardScanpy.run(input_dict_1,target_objs = ['X_from'])
    input_dict_1['w'].index = input_dict_1['w']['cell_type']
    
    mapping_df = input_dict_1['T'].copy()
    result_tg2integrate[noise] = (mapping_df)

    ad_sc_mapping = ad_sc_paired.copy()
    ad_sp_mapping = ad_sp.copy()
    
    tan2.pp_adatas(ad_sc_mapping, ad_sp_mapping)

    ad_map = tan2.map_cells_to_space(ad_sc_mapping, ad_sp_mapping,
        mode="cells",
        device='cuda:0',
        density_prior='uniform',
    )

    result_tg2cell[noise] = (ad_map.to_df().T)

    ad_sc_mapping = ad_sc_paired.copy()
    ad_sp_mapping = ad_sp.copy()
    
    
    tan.pp_adatas(ad_sc_mapping, ad_sp_mapping)
    ad_map = tan.map_cells_to_space(ad_sc_mapping, ad_sp_mapping,
        mode="cells",
        device='cuda:0',
        density_prior='uniform',
    )

    result_tg1cell[noise] = (ad_map.to_df().T)

    
## spaOTsc
    ad_sc_mapping = ad_sc_paired.copy()
    ad_sp_mapping = ad_sp.copy()
    
    input_dict_1 = tg.met.utils.adatas_to_input({'from':ad_sc_mapping, 'to':ad_sp_mapping}, # provide the data to be used
                                                    categorical_labels={'from':[label_col]}, # include cluster labels in the design matrix
                                                  )
    
    wf_spaOT_setup = {'pp' : {'method': tg.met.pp.StandardSpaOTsc},
                    'map':{'method': tg.met.map.SpaOTscMap,
                          'params' : {'num_epochs':1000, 'genes': None}},

                      'pred': {'method':tg.met.pred_methods.MoscotPred},

                   }

    wf_spaOT = Workflow(wf_spaOT_setup)
    wf_spaOT.run(input_dict_1)

    result_spaotsc[noise] = input_dict_1['T']

## MOSCOT
    
    ad_sc_mapping = ad_sc_paired.copy()
    ad_sp_mapping = ad_sp.copy()
    
    input_dict_1 = tg.met.utils.adatas_to_input({'from':ad_sc_mapping, 'to':ad_sp_mapping}, # provide the data to be used
                                                    categorical_labels={'from':[label_col]}, # include cluster labels in the design matrix
                                                  )

    wf_moscot_setup = {'pp' : {'method': tg.met.pp.StandardMoscot},
        'map':{'method': tg.met.map.MoscotMap,
             'params' : {'num_epochs':1000, 'genes': None}},
        'pred': {'method':tg.met.pred_methods.MoscotPred},

       }

    wf_moscot = Workflow(wf_moscot_setup)
    wf_moscot.run(input_dict_1)

    result_moscot[noise] = input_dict_1['T']


INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 20068 genes and uniform density_prior in clusters mode...
INFO:root:Printing scores every 100 epochs.


Set Solid Seed
Set Solid Seed
Score: 0.927, KL reg: 3.205, Entropy reg: -7.301
Score: 0.974, KL reg: 3.178, Entropy reg: -5.796
Score: 0.975, KL reg: 3.178, Entropy reg: -5.634
Score: 0.975, KL reg: 3.178, Entropy reg: -5.580
Score: 0.975, KL reg: 3.178, Entropy reg: -5.555
Score: 0.976, KL reg: 3.178, Entropy reg: -5.543
Score: 0.976, KL reg: 3.178, Entropy reg: -5.540
Score: 0.976, KL reg: 3.178, Entropy reg: -5.532
Score: 0.976, KL reg: 3.178, Entropy reg: -5.528
Score: 0.976, KL reg: 3.178, Entropy reg: -5.525


INFO:root:Renormalizing Single cell data
INFO:root:Begin training with 20068 genes and uniform density_prior in cells mode after renormalization
INFO:root:Printing scores every 100 epochs.


Set Solid Seed
Set Solid Seed
Score: 0.936, KL reg: 0.001, Entropy reg: -4132.243
Score: 0.997, KL reg: 0.000, Entropy reg: -521.425
Score: 0.998, KL reg: 0.000, Entropy reg: -411.384
Score: 0.998, KL reg: 0.000, Entropy reg: -382.132
Score: 0.998, KL reg: 0.000, Entropy reg: -369.306
Score: 0.998, KL reg: 0.000, Entropy reg: -361.381
Score: 0.998, KL reg: 0.000, Entropy reg: -358.768
Score: 0.998, KL reg: 0.000, Entropy reg: -355.380
Score: 0.998, KL reg: 0.000, Entropy reg: -353.432


INFO:root:Saving results..


...

Score: 0.935, KL reg: 0.001, Entropy reg: -4132.243
Score: 0.998, KL reg: 0.000, Entropy reg: -480.119
Score: 0.998, KL reg: 0.000, Entropy reg: -383.662
Score: 0.998, KL reg: 0.000, Entropy reg: -354.646
Score: 0.998, KL reg: 0.000, Entropy reg: -342.843
Score: 0.998, KL reg: 0.000, Entropy reg: -336.384
Score: 0.998, KL reg: 0.000, Entropy reg: -331.145
Score: 0.998, KL reg: 0.000, Entropy reg: -329.991
Score: 0.998, KL reg: 0.000, Entropy reg: -327.414
Score: 0.998, KL reg: 0.000, Entropy reg: -326.598


INFO:root:Saving results..
INFO:root:20068 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:20069 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 20068 genes and uniform density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.478, KL reg: 0.001
Score: 0.994, KL reg: 0.002
Score: 0.994, KL reg: 0.002
Score: 0.995, KL reg: 0.002
Score: 0.995, KL reg: 0.002
Score: 0.995, KL reg: 0.002
Score: 0.995, KL reg: 0.002
Score: 0.995, KL reg: 0.002
Score: 0.995, KL reg: 0.002
Score: 0.995, KL reg: 0.002


INFO:root:Saving results..


INFO     Computing pca with `n_comps=30` for `xy` using `adata.X`                                                  
INFO     Normalizing spatial coordinates of `x`.                                                                   
INFO     Solving `1` problems                                                                                      
INFO     Solving problem OTProblem[stage='prepared', shape=(100, 1002)].                                           


In [38]:
for i in range(1, len(noise_level)):    
    noise = noise_level[i]

    ad_sc_mapping = ad_sc_paired.copy()
    ad_sp_mapping = ad_sp.copy()
    ad_sp_mapping.X = ad_sp_mapping.layers[noise]
    
    label_col = label_used
    input_dict_1 = tg.met.utils.adatas_to_input({'from':ad_sc_mapping, 'to':ad_sp_mapping}, # provide the data to be used
                                                    categorical_labels={'from':[label_col]}, # include cluster labels in the design matrix
                                                  )
    argmax_wf = {#'pp' : dict(method = tg.methods.preprocess.StandardTangramV1),
                     'map': dict(method = tg.met.map_methods.ArgMaxCorrMap),
                    }
    
    #argmax_wf = dict(method = tg.met.map_methods.ArgMaxCorrMap)
    
    tg.met.workflows.Workflow(argmax_wf).run(input_dict_1)
    result_argmax[noise] = input_dict_1['T'].copy()

    ad_sc_mapping = ad_sc_paired.copy()
    ad_sp_mapping = ad_sp.copy()
    ad_sp_mapping.X = ad_sp_mapping.layers[noise]
    
    input_dict_1 = tg.met.utils.adatas_to_input({'from':ad_sc_mapping, 'to':ad_sp_mapping}, # provide the data to be used
                                                    categorical_labels={'from':[label_col]}, # include cluster labels in the design matrix
                                                  )
    random_wf = {#'pp' : dict(method = tg.methods.preprocess.StandardTangramV1),
                     'map': dict(method = tg.met.map_methods.RandomMap),
                    }
    
    #argmax_wf = dict(method = tg.met.map_methods.ArgMaxCorrMap)
    
    tg.met.workflows.Workflow(random_wf).run(input_dict_1)
    result_random[noise] = input_dict_1['T'].copy()


    
    ad_sc_mapping = ad_sc_paired.copy()
    ad_sp_mapping = ad_sp.copy()
    ad_sp_mapping.X = ad_sp_mapping.layers[noise]
    
    label_col = label_used
    input_dict_1 = tg.met.utils.adatas_to_input({'from':ad_sc_mapping, 'to':ad_sp_mapping}, # provide the data to be used
                                                    categorical_labels={'from':[label_col]}, # include cluster labels in the design matrix
                                                  )
    
    tg.met.pp.StandardTangramV2.run(input_dict_1)
    
    map_res_1 = tg.met.map_methods.TangramV2Map.run(input_dict_1,
                                                num_epochs = 1000,
                                                density_prior='uniform',
                                             )
    
    input_dict_1.update(map_res_1)
    tg.met.pp.StandardScanpy.run(input_dict_1,target_objs = ['X_from'])
    input_dict_1['w'].index = input_dict_1['w']['cell_type']
    
    mapping_df = input_dict_1['T'].copy()
    result_tg2integrate[noise] = (mapping_df)

    ad_sc_mapping = ad_sc_paired.copy()
    ad_sp_mapping = ad_sp.copy()
    ad_sp_mapping.X = ad_sp_mapping.layers[noise]
    
    tan2.pp_adatas(ad_sc_mapping, ad_sp_mapping)

    ad_map = tan2.map_cells_to_space(ad_sc_mapping, ad_sp_mapping,
        mode="cells",
        device='cuda:0',
        density_prior='uniform',
    )

    result_tg2cell[noise] = (ad_map.to_df().T)

    ad_sc_mapping = ad_sc_paired.copy()
    ad_sp_mapping = ad_sp.copy()
    ad_sp_mapping.X = ad_sp_mapping.layers[noise]
    
    
    tan.pp_adatas(ad_sc_mapping, ad_sp_mapping)
    ad_map = tan.map_cells_to_space(ad_sc_mapping, ad_sp_mapping,
        mode="cells",
        device='cuda:0',
        density_prior='uniform',
    )

    result_tg1cell[noise] = (ad_map.to_df().T)

    
## spaOTsc
    ad_sc_mapping = ad_sc_paired.copy()
    ad_sp_mapping = ad_sp.copy()
    ad_sp_mapping.X = ad_sp_mapping.layers[noise]
    
    input_dict_1 = tg.met.utils.adatas_to_input({'from':ad_sc_mapping, 'to':ad_sp_mapping}, # provide the data to be used
                                                    categorical_labels={'from':[label_col]}, # include cluster labels in the design matrix
                                                  )
    
    wf_spaOT_setup = {'pp' : {'method': tg.met.pp.StandardSpaOTsc},
                    'map':{'method': tg.met.map.SpaOTscMap,
                          'params' : {'num_epochs':1000, 'genes': None}},

                      'pred': {'method':tg.met.pred_methods.MoscotPred},

                   }

    wf_spaOT = Workflow(wf_spaOT_setup)
    wf_spaOT.run(input_dict_1)

    result_spaotsc[noise] = input_dict_1['T']

## MOSCOT
    
    ad_sc_mapping = ad_sc_paired.copy()
    ad_sp_mapping = ad_sp.copy()
    ad_sp_mapping.X = ad_sp_mapping.layers[noise]
    
    input_dict_1 = tg.met.utils.adatas_to_input({'from':ad_sc_mapping, 'to':ad_sp_mapping}, # provide the data to be used
                                                    categorical_labels={'from':[label_col]}, # include cluster labels in the design matrix
                                                  )

    wf_moscot_setup = {'pp' : {'method': tg.met.pp.StandardMoscot},
        'map':{'method': tg.met.map.MoscotMap,
             'params' : {'num_epochs':1000, 'genes': None}},
        'pred': {'method':tg.met.pred_methods.MoscotPred},

       }

    wf_moscot = Workflow(wf_moscot_setup)
    wf_moscot.run(input_dict_1)

    result_moscot[noise] = input_dict_1['T']


INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 19155 genes and uniform density_prior in clusters mode...
INFO:root:Printing scores every 100 epochs.


Set Solid Seed
Set Solid Seed
Score: 0.913, KL reg: 3.205, Entropy reg: -7.301
Score: 0.961, KL reg: 3.178, Entropy reg: -5.793
Score: 0.961, KL reg: 3.178, Entropy reg: -5.626
Score: 0.962, KL reg: 3.178, Entropy reg: -5.571
Score: 0.962, KL reg: 3.178, Entropy reg: -5.546
Score: 0.962, KL reg: 3.178, Entropy reg: -5.532
Score: 0.962, KL reg: 3.178, Entropy reg: -5.522
Score: 0.962, KL reg: 3.178, Entropy reg: -5.516
Score: 0.962, KL reg: 3.178, Entropy reg: -5.512
Score: 0.962, KL reg: 3.178, Entropy reg: -5.511


INFO:root:Renormalizing Single cell data
INFO:root:Begin training with 19155 genes and uniform density_prior in cells mode after renormalization
INFO:root:Printing scores every 100 epochs.


Set Solid Seed
Set Solid Seed
Score: 0.921, KL reg: 0.001, Entropy reg: -4132.243
Score: 0.983, KL reg: 0.000, Entropy reg: -511.684
Score: 0.983, KL reg: 0.000, Entropy reg: -398.187
Score: 0.983, KL reg: 0.000, Entropy reg: -367.432
Score: 0.983, KL reg: 0.000, Entropy reg: -355.715
Score: 0.983, KL reg: 0.000, Entropy reg: -347.815
Score: 0.983, KL reg: 0.000, Entropy reg: -343.872
Score: 0.983, KL reg: 0.000, Entropy reg: -340.208
Score: 0.983, KL reg: 0.000, Entropy reg: -338.987


INFO:root:Saving results..


...

Score: 0.439, KL reg: 0.001, Entropy reg: -4132.243
Score: 0.459, KL reg: 0.000, Entropy reg: -3430.189
Score: 0.460, KL reg: 0.000, Entropy reg: -3391.178
Score: 0.460, KL reg: 0.000, Entropy reg: -3382.131
Score: 0.460, KL reg: 0.000, Entropy reg: -3378.244
Score: 0.460, KL reg: 0.000, Entropy reg: -3376.303
Score: 0.460, KL reg: 0.000, Entropy reg: -3375.042
Score: 0.460, KL reg: 0.000, Entropy reg: -3374.319
Score: 0.460, KL reg: 0.000, Entropy reg: -3373.792
Score: 0.460, KL reg: 0.000, Entropy reg: -3373.389


INFO:root:Saving results..
INFO:root:20068 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:20069 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 20068 genes and uniform density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.569, KL reg: 0.001
Score: 0.682, KL reg: 0.004
Score: 0.682, KL reg: 0.004
Score: 0.682, KL reg: 0.004
Score: 0.682, KL reg: 0.004
Score: 0.682, KL reg: 0.004
Score: 0.682, KL reg: 0.004
Score: 0.682, KL reg: 0.004
Score: 0.682, KL reg: 0.004
Score: 0.682, KL reg: 0.004


INFO:root:Saving results..


INFO     Computing pca with `n_comps=30` for `xy` using `adata.X`                                                  
INFO     Normalizing spatial coordinates of `x`.                                                                   
INFO     Solving `1` problems                                                                                      
INFO     Solving problem OTProblem[stage='prepared', shape=(100, 1002)].                                           


In [39]:
result_df = pd.DataFrame(index = noise_level)
for noise in noise_level:
    result_df.loc[noise, 'tg1_cell'] = jaccard_score((result_tg1cell[noise] == result_tg1cell[noise].max()).astype(int).values.flatten(), ground_truth.values.flatten())
    result_df.loc[noise, 'tg2_cell'] = jaccard_score((result_tg2cell[noise] == result_tg2cell[noise].max()).astype(int).values.flatten(), ground_truth.values.flatten() )
    result_df.loc[noise, 'tg2_integrate'] = jaccard_score((result_tg2integrate[noise] == result_tg2integrate[noise].max()).astype(int).values.flatten(), ground_truth.values.flatten() )
    result_df.loc[noise, 'moscot'] = jaccard_score((result_moscot[noise] == result_moscot[noise].max()).astype(int).values.flatten(), ground_truth.values.flatten() )
    result_df.loc[noise, 'spaotsc'] = jaccard_score((result_spaotsc[noise] == result_spaotsc[noise].max()).astype(int).values.flatten(), ground_truth.values.flatten() )
    result_df.loc[noise, 'argmax'] = jaccard_score((result_argmax[noise] == result_argmax[noise].max()).astype(int).values.flatten(), ground_truth.values.flatten() )
    result_df.loc[noise, 'random'] = jaccard_score((result_random[noise] == result_random[noise].max()).astype(int).values.flatten(), ground_truth.values.flatten() )

In [40]:
result_df

,tg1_cell,tg2_cell,tg2_integrate,moscot,spaotsc,argmax,random
no_noise,0.998006,0.970501,0.968566,0.038322,0.008556,0.083829,0.003003
resample_0.0,0.998006,0.958944,0.957031,0.036730,0.008556,0.085590,0.003003
resample_0.1,0.964706,0.982196,0.980237,0.036730,0.008556,0.083243,0.003003
resample_0.2,0.841912,0.986125,0.988095,0.037804,0.008556,0.085590,0.003003
resample_0.3,0.596813,0.974384,0.980237,0.038880,0.008556,0.079160,0.003003
resample_0.4,0.351315,0.939981,0.962782,0.039958,0.008556,0.081489,0.003003
resample_0.5,0.213075,0.876404,0.899526,0.030334,0.008556,0.084416,0.003003
resample_0.6,0.115813,0.760984,0.779751,0.026639,0.008556,0.080323,0.003003
resample_0.7,0.064259,0.585443,0.607057,0.017776,0.008556,0.083243,0.003003
resample_0.8,0.027692,0.363265,0.403361,0.013145,0.008556,0.071658,0.003003


In [ ]:
result_df.to_csv('/gstore/data/resbioai/tangram2_data/telegraph/res/mapping/100_spot/' + patient + '_result.csv')